In [1]:
import numpy as np 
import pandas as pd
import seaborn as sb 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from datetime import datetime
import os

import nltk 
print(nltk.__version__)
from nltk.tokenize import * 
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem.porter import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

import re 
from IPython.display import HTML


3.7


In [2]:
with open('/home/cyrus/Desktop/workspace/cps3235/data/data/20210105_prop_for_sale_tom_classifieds.html') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    try:
        date = soup.find("h2", {"class": "classified_date default_top_margin"}).text
    except:
        date = soup.find("h3").text
    
    date  = datetime.strptime(date, '%A, %B %d, %Y').date()

date


datetime.date(2021, 1, 5)

In [133]:
def extract_properties(filename):
    print("Parsing file: " + filename)
    with open(filename) as f:
        contents = f.read()
        properties = []

        soup = BeautifulSoup(contents, 'lxml')
        list_of_ul= soup.find_all("ul",{ "class" : "classified_list"})
        for ul in list_of_ul:
            li_list = ul.find_all("li")
            for child in li_list:
                properties.append(child.text)
        
        if len(properties) == 0:
            list_of_ul = soup.find_all("li", {"class" : "ca-Classifieds_PostListItem"})
            for child in list_of_ul:
                properties.append(child.text)

        try:
            date = soup.find("h2", {"class": "classified_date default_top_margin"}).text
        except:
            date = soup.find("h3").text

        date  = datetime.strptime(date, '%A, %B %d, %Y').date()

        for idx, property in enumerate(properties): 
            properties[idx] = property.replace('\nProperty For Sale\n', '')

            if 'website' in property:
                properties.pop(idx)
                idx-=1   
        
        properties.pop(0)

        properties = [property.replace('\n', '') for property in properties]

        f.close()
        return properties, date

In [134]:
# def extract_properties(filename):
#     print('Parsing: '+ filename)
#     with open(filename, 'r') as f:
#         contents = f.read()
#         soup = BeautifulSoup(contents, 'lxml')

#         list_of_a = soup.find_all("a",{"href":"http://www.timesofmalta.com/classifieds/property-for-sale"})

#         if(len(list_of_a) == 0):
#             print(list_of_a)
#             list_of_a = soup.find("ul",{ "class" "classified_list"})
        

#         date = soup.find("h2", {"class": "classified_date default_top_margin"}).text

#         date  = datetime.strptime(date, '%A, %B %d, %Y').date()
        
#         properties = []
#         for element in list_of_a:
#             properties.append(element.parent.parent.text)
            
#         # for idx, property in enumerate(properties): 
#         #     properties[idx] = property.replace('\nProperty For Sale\n', '')

#         #     if 'website' in property:
#         #         properties.pop(idx)
#         #         idx-=1

#         # for idx, property in enumerate(properties): 
#         #     properties[idx] = property.replace('\nProperty For Sale\n', '')
#         #     properties[idx] = property.replace(':', '.')
        
#         # properties.pop(0)

#     f.close()
#     return properties, date


In [135]:
extract_properties('20150423_prop_for_sale_tom_classifieds.html')

Parsing file: 20150423_prop_for_sale_tom_classifieds.html


(['GOZO, GĦAJNSIELEM. An ideal residential plot of land for investment or for a good sized terraced house, situated in a three story-height zone. €185,000. Phone 9947 6959.',
  'GOZO, NADUR: 3 double bedroom house of character with lots of traditional features and a small pool. Worth viewing. €293,000. Phone 7947 0419.',
  'GOZO, VICTORIA: Spacious ground floor maisonette ideal for first time buyers or retired couple. Superb property in very good condition. €145,000. Phone 9949 2695.',
  'MARSASCALA. Finished two bedroom apartment close to St Thomas bay. €85,000. Phone 2163 2919.',
  'QAWRA. Apartment, three bedrooms, furnished, open plan, balconies, freehold. €140,000 (Lm60,102 ). Phone 9905 6212.',
  'QAWRA. Bargain. One / two / three bedroom maisonettes, apartments and penthouses being offered fully finished from €75,500 on plan with open views. Phone owner 9922 0023. ',
  'SIĠĠIEWI. Three bedroom ground floor maisonette. No steps. €120,000 negotiable. Phone owner 9982 6148. ',
  "S

In [136]:
df = pd.DataFrame()
lemmatizer = WordNetLemmatizer()

tokenizer = RegexpTokenizer(r'\w+')

location = []
property_type = []
furnished = []
pool = []
balcony = [] 
garage = []
sea_view = []
newly_built = []
no_of_bedrooms = []
double_bedroom = []
price = []

def is_furnished(words):
    if 'finished' in words or 'furnished' in words:
        furnished.append(1)
    elif 'refubish' in words:
        furnished.append(0)
    else: 
        furnished.append(np.nan)

def property_types(word):
    switcher = {
        "apartment": 0,
        "land": 1, 
        "house": 2,
        "guesthouse": 2, 
        "penthouse": 3, 
        "bungalow": 4, 
        "farmhouse": 5,
        "restaurant": 6
    }

    return switcher.get(word)

def did_len_change(initial_len, final_len):
    if initial_len == final_len:
        return False
    else:
        return True

def get_property_type(words):
    initial_length = len(property_type)
    for idx, word in enumerate(words):
        type = property_types(word)
        if type is not None:
            property_type.append(type)
            break
        elif idx == len(words)-1:
            property_type.append(np.nan)

    if did_len_change(initial_length, len(property_type)) is False:
        print("External error")
        print(words)

def is_newly_built(words):
    if 'new' in words:
        newly_built.append(1)
    else:
        newly_built.append(0)


def has_sea_view(words):
    if 'sea' in words:
        sea_view.append(1)
    else:
        sea_view.append(0)


def contains_pool(words):
    if 'pool' in words:
        pool.append(1)
    else:
        pool.append(0)
        
def contains_garage(words):
    if 'garage' in words:
        garage.append(1)
    else:
        garage.append(0)

def has_balconies(words):
    if 'balcony' in words:
        balcony.append(1)
    else:
        balcony.append(0)

def word_to_number(word):
    numbers = {
        'one': 1,
        'two': 2,
        'three': 3,
        'four': 4,
        'five': 5,
        'six': 6,
        'seven': 7,
        'eight': 8,
        'nine': 9,
        'zero': 0
    }

    return numbers.get(word)

def is_double_bedroom(words):
    if 'double' in words:
        double_bedroom.append(1)
    else:
        double_bedroom.append(0)
  
def get_no_of_bedrooms(words):
    init_len = len(no_of_bedrooms)
    if 'bedroom' in words:
        bedroom_idx = words.index('bedroom') - 1
        if words[bedroom_idx] == 'double' or words[bedroom_idx] == 'large':
            bedroom_count = words[bedroom_idx - 1]
        else:
            try:
                bedroom_count = int(words[bedroom_idx])
            except:
                no_of_bedrooms.append(np.nan)
                return
        
        no_of_bedrooms.append(bedroom_count)
    else:
        no_of_bedrooms.append(np.nan)
    

    if did_len_change(init_len, len(no_of_bedrooms)) is False:
        print(words)

def extract_price(str_to_extract):
    pattern = re.compile(r'(€[0-9]+,[0-9]+)')
    try:
        match = pattern.search(str_to_extract).group(0).replace('€', '')
    except:
        return -1
    return int(match.replace(',', ''))

def my_tokenizer(str_to_split):
     start_idx = str_to_split.find('.') + 1
     end_idx = str_to_split.find('Phone')
     property = str_to_split[start_idx:end_idx-2]
     if len(property) != 0:
          location.append(str_to_split[:start_idx])
     
     return property


In [137]:
# features: finished, no. of bedrooms, type of property, pool, balcony, garage, sea-view, newly-built 
def extract_features_to_df(properties, date):
    for property in properties:
        info = my_tokenizer(property)
        info = info.lower()

        if len(info) == 0: 
            continue 

        if extract_price(info) != -1:
            price.append(extract_price(info))
        else:
            price.append(np.nan)

        words = tokenizer.tokenize(info)
        filtered_words = []
        filtered_words = [word for word in words if word not in stopwords.words('english')]
        filtered_words = [lemmatizer.lemmatize(word) for word in filtered_words]
        filtered_words = [word_to_number(word) if word_to_number(word) is not None else word for word in filtered_words]


        is_furnished(filtered_words)
        get_property_type(filtered_words)
        is_newly_built(filtered_words)
        has_sea_view(filtered_words)
        contains_pool(filtered_words)
        contains_garage(filtered_words)
        has_balconies(filtered_words)
        is_double_bedroom(filtered_words)
        get_no_of_bedrooms(filtered_words)
        
    # TODO: Assert length of each is equal.
    # print(len(location))
    # print(len(property_type))
    # print(len(furnished))
    # print(len(pool))
    # print(len(garage))
    # print(len(sea_view))
    # print(len(newly_built))
    # print(len(no_of_bedrooms))
    # print(len(double_bedroom))
    # print(len(price))
    temp_df = pd.DataFrame()
    temp_df['location'] = location
    temp_df['Property type'] = property_type
    temp_df['Furnished'] = furnished
    temp_df['Pool'] = pool
    temp_df['Garage'] = garage
    temp_df['Near sea'] = sea_view
    temp_df['Newly built'] = newly_built
    temp_df['double bedroom'] = double_bedroom
    temp_df['No. of bedrooms'] = no_of_bedrooms
    temp_df['Date'] = date
    temp_df['Price'] = price

    return temp_df

In [138]:
dir = os.getcwd()
os.chdir(dir)
for filename in os.listdir(dir):
    if os.path.getsize(filename) == 0:
        continue
    
    properties, date = extract_properties(filename)
    temp_df = extract_features_to_df(properties, date)
    df = pd.concat([df, temp_df], axis=0)
    location = []
    property_type = []
    furnished = []
    pool = []
    balcony = [] 
    garage = []
    sea_view = []
    newly_built = []
    no_of_bedrooms = []
    double_bedroom = []
    price = []
    
df

Parsing file: 20161009_prop_for_sale_tom_classifieds.html
Parsing file: 20151026_prop_for_sale_tom_classifieds.html
Parsing file: 20180704_prop_for_sale_tom_classifieds.html
Parsing file: 20210105_prop_for_sale_tom_classifieds.html
Parsing file: 20180815_prop_for_sale_tom_classifieds.html
Parsing file: 20180528_prop_for_sale_tom_classifieds.html
Parsing file: 20160404_prop_for_sale_tom_classifieds.html
Parsing file: 20210201_prop_for_sale_tom_classifieds.html
Parsing file: 20150617_prop_for_sale_tom_classifieds.html
Parsing file: 20220301_prop_for_sale_tom_classifieds.html
Parsing file: 20170918_prop_for_sale_tom_classifieds.html
Parsing file: 20160229_prop_for_sale_tom_classifieds.html
Parsing file: 20180829_prop_for_sale_tom_classifieds.html
Parsing file: 20191115_prop_for_sale_tom_classifieds.html
Parsing file: 20161115_prop_for_sale_tom_classifieds.html
Parsing file: 20170523_prop_for_sale_tom_classifieds.html
Parsing file: 20220704_prop_for_sale_tom_classifieds.html
Parsing file: 

,location,Property type,Furnished,Pool,Garage,Near sea,Newly built,double bedroom,No. of bedrooms,Date,Price
0,"Property For SaleATTARD, Misraħ Kola area.",4.0,NaN,1,1,0,0,0,4,2016-10-09,595000.0
1,"ATTARD, Misraħ Kola.",NaN,NaN,0,0,0,0,0,NaN,2016-10-09,550000.0
2,"ATTARD, Tal-Mirakli area.",NaN,NaN,0,1,0,0,0,3,2016-10-09,240000.0
3,ATTARD.,NaN,1.0,0,0,0,0,0,3,2016-10-09,260000.0
4,ATTARD.,3.0,NaN,0,1,0,0,0,2,2016-10-09,250000.0
...,...,...,...,...,...,...,...,...,...,...,...
140,100% FOCUSED on quality properties.,NaN,NaN,0,0,0,0,0,NaN,2022-10-11,NaN
141,100% FOCUSED on quality properties.,NaN,NaN,0,0,0,0,0,NaN,2022-10-11,NaN
142,100% FOCUSED on quality properties.,NaN,NaN,0,0,0,0,0,NaN,2022-10-11,NaN
143,"GOZO, XLENDI.",0.0,NaN,0,0,1,0,0,2.0,2022-10-11,265000.0


In [139]:
df.to_csv('properties.csv', index=False)